In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ["HF_TOKEN"]=os.getenv('HF_TOKEN')
os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "rag-pdf-project"


In [10]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.parsers import RapidOCRBlobParser


file_path="./data/sample.pdf"
loader=PyMuPDFLoader(
    file_path,
    mode="page",
    images_inner_format="markdown-img",
    images_parser=RapidOCRBlobParser(),
    #extract_tables="markdown"      --- u can use this when your pdf contains real tables
)


docs=loader.load()

In [11]:
import pprint
pprint.pp(docs[215].page_content)

('Notes to the Group Financial Statements continued\n'
 '33. Group companies\n'
 'In accordance with Section 409 of the Companies Act 2006, a full list of '
 'entities in which the Group has an interest of greater than or equal \n'
 'to 20%, the registered office and effective percentage of equity owned as at '
 '31 December 2023 are disclosed below. Unless otherwise stated, \n'
 'the ownership interest disclosed comprises either ordinary shares, '
 'certificated or un-certificated membership interests which are indirectly \n'
 'held by InterContinental Hotels Group PLC.\n'
 'Fully owned subsidiaries\n'
 '10000 Champion Acquisition LLC (k)\n'
 '24th Street Operator Sub, LLC (k)\n'
 '2250 Blake Street Hotel, LLC (k)\n'
 '36th Street IHG Sub, LLC (k)\n'
 '426 Main Ave, LLC. (k)\n'
 '46 Nevins Street Associates, LLC (k)\n'
 'Alpha Kimball Hotel, LLC (k)\n'
 'Asia Pacific Holdings Limited (n)\n'
 'Barclay Operating Corp. (k)\n'
 'BHMC Canada Inc. (o)\n'
 'BHR Holdings B.V. (p)\n'
 'BHR Pac

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50,separators=["\n\n", "\n" , ".", "?",","," "])
texts=text_splitter.split_documents(docs)

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\rohit\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Extract raw text from Document objects
text_chunks = [doc.page_content for doc in texts]

# Embed only the text content
embeddings = embeddings.embed_documents(text_chunks)

print(f"✅ Number of embeddings: {len(embeddings)}")
print(f"✅ Shape of first embedding: {len(embeddings[0])}")


KeyboardInterrupt: 

In [15]:
from langchain.schema import Document

docs = [Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadatas)]

NameError: name 'metadatas' is not defined

In [59]:
from langchain_community.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import os, time

# Load credentials securely
KEY = os.getenv('ZILLIZ_API_KEY')
URL = os.getenv('ZILLIZ_URI')

# Ensure your credentials are loaded
assert KEY is not None, "ZILLIZ_API_KEY not set"
assert URL is not None, "ZILLIZ_URI not set"

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Index configurations
index_types = {
    "flat": {"index_type": "flat", "metric_type": "L2"},
    "hnsw": {"index_type": "hnsw", "metric_type": "L2", "params": {"M": 16, "efConstruction": 200}},
    "ivf": {"index_type": "ivf_flat", "metric_type": "L2", "params": {"nlist": 100}},
}

# Loop through each index type and store vectors
for name, index_param in index_types.items():
    print(f"🚀 Creating collection: rag_{name}")
    start = time.time()

    vector_store = Milvus.from_documents(
        documents=docs,
        embedding=embedding_model,
        connection_args={"uri": URL, "token": KEY},
        collection_name=f"rag1_{name}",
        index_params=index_param
    )

    print(f"✅ Index '{name}' created in {time.time() - start:.2f} seconds\n")


🚀 Creating collection: rag_flat


2025-07-04 15:09:17,388 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: flat])>, <Time:{'RPC start': '2025-07-04 15:09:17.079699', 'RPC error': '2025-07-04 15:09:17.388274'}> (decorators.py:140)


✅ Index 'flat' created in 55.76 seconds

🚀 Creating collection: rag_hnsw


2025-07-04 15:10:15,312 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: hnsw])>, <Time:{'RPC start': '2025-07-04 15:10:15.038201', 'RPC error': '2025-07-04 15:10:15.312383'}> (decorators.py:140)


✅ Index 'hnsw' created in 58.04 seconds

🚀 Creating collection: rag_ivf


2025-07-04 15:11:21,088 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=1100, message=invalid parameter[expected=valid index][actual=invalid index type: ivf_flat])>, <Time:{'RPC start': '2025-07-04 15:11:20.781385', 'RPC error': '2025-07-04 15:11:21.088607'}> (decorators.py:140)


✅ Index 'ivf' created in 64.53 seconds



In [60]:
collection_map = {
    "flat": "rag1_flat",
    "hnsw": "rag1_hnsw",
    "ivf": "rag1_ivf"
}


In [94]:
def retrieve_with_timing(index_name: str, query: str, k: int = 5):
    vs = Milvus(
        embedding_function=embedding_model,
        connection_args={"uri": URL, "token": KEY},
        collection_name=collection_map[index_name]
    )

    retriever = vs.as_retriever(search_kwargs={"k": k})

    start = time.time()
    results = retriever.get_relevant_documents(query)
    duration = time.time() - start

    return results, duration


In [102]:
query = "What are Key performance measures and non-GAAP measures"

retrieval_logs = {}

for index in collection_map:
    results, duration = retrieve_with_timing(index, query)
    retrieval_logs[index] = {
        "results": results,
        "time": duration
    }


In [101]:
for index, log in retrieval_logs.items():
    print(f"📊 {index.upper()} — Time: {log['time']:.4f}s")
    print("🔹 Top Result Preview:", log["results"][0].page_content[:200], "\n")


📊 FLAT — Time: 0.2771s
🔹 Top Result Preview: (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non-GAAP measures are either not defined under 
IFRS or are adjusted IFRS figures. F 

📊 HNSW — Time: 0.3079s
🔹 Top Result Preview: (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non-GAAP measures are either not defined under 
IFRS or are adjusted IFRS figures. F 

📊 IVF — Time: 0.4123s
🔹 Top Result Preview: (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non-GAAP measures are either not defined under 
IFRS or are adjusted IFRS figures. F 



In [100]:
def precision_at_k(results, expected_keywords, k=5):
    hits = 0
    for doc in results[:k]:
        content = doc.page_content.lower()
        if any(keyword.lower() in content for keyword in expected_keywords):
            hits += 1
    return hits / k


In [103]:
# Define what you expect in a correct answer
expected_keywords = ["Non-GAAP","measures","performance","figures"]

# Store precision scores
for index, log in retrieval_logs.items():
    precision = precision_at_k(log["results"], expected_keywords, k=10)
    retrieval_logs[index]["precision@5"] = precision
    print(f"🎯 {index.upper()} — Precision@5: {precision:.2f} — Time: {log['time']:.4f}s")


🎯 FLAT — Precision@5: 0.50 — Time: 0.3103s
🎯 HNSW — Precision@5: 0.50 — Time: 0.2894s
🎯 IVF — Precision@5: 0.50 — Time: 0.3152s


In [7]:
import matplotlib.pyplot as plt

indexes = list(retrieval_logs.keys())
times = [retrieval_logs[idx]["time"] for idx in indexes]
precisions = [retrieval_logs[idx]["precision@5"] for idx in indexes]

plt.figure(figsize=(8, 4))
plt.bar(indexes, precisions, alpha=0.6, label="Precision@5", color="green")
plt.plot(indexes, times, marker='o', label="Retrieval Time (s)", color="blue")
plt.legend()
plt.title("Precision@5 vs Retrieval Time")
plt.ylabel("Score / Time (s)")
plt.xlabel("Index Type")
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'retrieval_logs' is not defined

In [106]:
def retrieve_with_mmr(index_name: str, query: str, k: int = 5, fetch_k: int = 20, lambda_mult: float = 0.5):
    print(f"\n🔁 MMR Re-ranking in: {index_name.upper()}")

    vs = Milvus(
        embedding_function=embedding_model,
        connection_args={"uri": URL, "token": KEY},
        collection_name=collection_map[index_name]
    )

    retriever = vs.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": k,
            "fetch_k": fetch_k,        # Number of initial candidates
            "lambda_mult": lambda_mult # Balance: 0.0 = max diversity, 1.0 = max relevance
        }
    )

    start = time.time()
    results = retriever.get_relevant_documents(query)
    duration = time.time() - start

    print(f"⏱️ MMR Retrieval Time: {duration:.4f} seconds")
    print("📄 MMR Top Result Preview:\n", results[0].page_content[:300], "\n")

    return results, duration


In [107]:
query = "What are Key performance measures and non-GAAP measures"

mmr_logs = {}

for index in collection_map:
    results, duration = retrieve_with_mmr(index, query)
    mmr_logs[index] = {
        "results": results,
        "time": duration
    }



🔁 MMR Re-ranking in: FLAT
⏱️ MMR Retrieval Time: 0.7518 seconds
📄 MMR Top Result Preview:
 (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non-GAAP measures are either not defined under 
IFRS or are adjusted IFRS figures. Further explanation in relation to these measures can be found on page 84 to 88 and reconciliations t 


🔁 MMR Re-ranking in: HNSW
⏱️ MMR Retrieval Time: 0.9001 seconds
📄 MMR Top Result Preview:
 (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non-GAAP measures are either not defined under 
IFRS or are adjusted IFRS figures. Further explanation in relation to these measures can be found on page 84 to 88 and reconciliations t 


🔁 MMR Re-ranking in: IVF
⏱️ MMR Retrieval Time: 1.1073 seconds
📄 MMR Top Result Preview:
 (described as Non-GAAP) are presented that are used internally by management as key measures to assess performance. Non

In [108]:
expected_keywords = ["Non-GAAP","measures","performance","figures"]

for index, log in mmr_logs.items():
    precision = precision_at_k(log["results"], expected_keywords)
    mmr_logs[index]["precision@5"] = precision
    print(f"🔁 MMR {index.upper()} — Precision@5: {precision:.2f} — Time: {log['time']:.4f}s")


🔁 MMR FLAT — Precision@5: 1.00 — Time: 0.7518s
🔁 MMR HNSW — Precision@5: 1.00 — Time: 0.9001s
🔁 MMR IVF — Precision@5: 1.00 — Time: 1.1073s


In [6]:
print("📊 MMR vs Default Precision@5\n")
for index in collection_map:
    original = retrieval_logs[index]["precision@5"]
    reranked = mmr_logs[index]["precision@5"]
    print(f"{index.upper()} — Base: {original:.2f}, MMR: {reranked:.2f}")



📊 MMR vs Default Precision@5



NameError: name 'collection_map' is not defined

In [114]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful assistant. Use the context below to answer the question.
If the context does not contain enough information, say so clearly.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""
)


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')


In [5]:
query = "What is the benefit of using HNSW for similarity search?"
context_docs = mmr_logs["hnsw"]["results"]

# Combine context
context = "\n\n".join([doc.page_content for doc in context_docs])

# Run Gemini through LLMChain
response = qa_chain.run({
    "context": context,
    "question": query
})

print("🧠 Gemini Answer:\n", response)


NameError: name 'mmr_logs' is not defined